In [1]:
ls

crosstalk_thoughts.ipynb                 performance_of_correction.png
crosstalk_vs_rate.jpg                    plots_from_jens/
foo.png                                  README.md
foo.svg                                  single_pe_spectra.hdf
formula_foo.png                          Untitled2.ipynb
gedankenexperiment_with_crosstalk.ipynb  Untitled.ipynb
papers/


In [3]:
import h5py

/home/dneise/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [7]:
df = pd.read_hdf(input_file_path, key='Baseline')
df.head()

ValueError: key must be provided when HDF5 file contains multiple datasets.

In [8]:
R = pd.read_hdf(input_file_path, key='Rate')

In [10]:
R

,,0,1,2,3,4,5,6,7,8,9,...,1430,1431,1432,1433,1434,1435,1436,1437,1438,1439
night,run_id,,,,,,,,,,,,,,,,,,,,,
2017-12-23,0,4.227833,4.439912,4.431358,3.935073,4.250155,3.70316,3.930467,4.410874,4.053284,4.082656,...,4.076224,3.943959,4.214595,3.945475,3.470208,4.052358,4.033941,3.73369,4.110238,3.89701


In [4]:
input_file_path = './single_pe_spectra.hdf'

In [5]:
with h5py.File(input_file_path) as f:
    keys = list(f.keys())
    print(*keys, sep=" | ")

Baseline | Chi2 | CoeffR | Crosstalk | FitProb | Gain | Noise | NormGain | Pxtalk | Rate | RelSigma | Signal


In [17]:
t_0 = 7e-9

In [19]:
mu = R.iloc[0][0] * 1e6 * t_0

In [22]:
mu = R * 1e6 * t_0

In [38]:
from scipy.stats import poisson
from scipy.special import factorial

In [34]:
P_mu = poisson.pmf(k=np.arange(0, 10)[:, None], mu=mu)

In [35]:
p = pd.read_hdf(input_file_path, key='Pxtalk')
nu = pd.read_hdf(input_file_path, key='CoeffR')
q = p * np.exp(-p)

In [39]:
def calc_Pn(N, q, nu):
    return (q*N)**(N-1) / (factorial(N-1))**nu

In [47]:
Pn = np.zeros((10, 1440))
for n in range(1, 11):
    Pn[n-1] = calc_Pn(n , q, nu)